<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center> 

_____

<a id='home'></a>

<a target="_blank" href="https://colab.research.google.com/github/CienciaDeDatosEspacial/merging/blob/main/Merging_DFs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Merging


Merging data sets need the following considerations:

* Merging is done on two data frames.
* You need a column in each data frame that share the same exact and unique values. The column names or titles need not be the same.
* The merged table shows by default the mutual coincidences; but you can also request the values not matched, which will help you detect possible extra cleaning.
* Pandas jargon uses a **left** and a **right** data frame: **left**.merge(**right**).

At this stage, let me use other data frames we prepared previously:

In [26]:
import pandas as pd

co2Link='https://github.com/CienciaDeDatosEspacial/dataSets/raw/main/co2.csv'
forestLink='https://github.com/CienciaDeDatosEspacial/dataSets/raw/main/forestRev.csv'

co2=pd.read_csv(co2Link)
forest=pd.read_csv(forestLink)

Remember the amount of rows of each DF:

In [27]:
co2.shape,forest.shape

((218, 4), (204, 3))

Also keep in mind the column names:

In [28]:
forest.columns,co2.columns

(Index(['Country', 'ForestRev_gdp', 'ForestRev_date'], dtype='object'),
 Index(['name', 'co2', 'co2_date', 'region'], dtype='object'))

Let me show you some merge approaches, but I will only show the amount of columns produced:

1. You keep only what is common in both key columns:

This is the default. The final rows will be the ones where the key values in each data frame match exactly. In this case, your count of rows will be at most the amount of rows of the smallest data frame.

In [29]:
# how many resulting rows after inner merging
co2.merge(forest,how='inner',left_on='name',right_on='Country').shape

(197, 7)

2. You keep all the keys from one data frame:

The final rows will be all the rows from the dataframe (here from the _left_). If a key values does not find a match, the key value is kept, but the columns will have missing values. In this case, your count of rows will be equal to the amount of rows of the data frame to the left. You can also use **right** so the same logic applies to the data frame to the right.



In [30]:
# how many resulting rows after left merging
co2.merge(forest,how='left',left_on='name',right_on='Country').shape

(218, 7)

3. You keep all the rows from both data frames:

In this case you will obtain all possible rows: the matched values, and the unmatched values from both data frames. You will also generate missing values. In this case, your count of rows will be at least the amount of rows of the data frame with the most rows.


In [31]:
# how many resulting rows after outer merging
co2.merge(forest,how='outer',left_on='name',right_on='Country').shape

(225, 7)

Why the different amount of rows? 

In [32]:
set(co2.name)-set(forest.Country)

{'ANTARCTICA',
 'BERMUDA',
 'BRITISH VIRGIN ISLANDS',
 'COOK ISLANDS',
 'ERITREA',
 'FALKLAND ISLANDS (ISLAS MALVINAS)',
 'FRENCH POLYNESIA',
 'GIBRALTAR',
 'JERSEY',
 'KOREA, NORTH',
 'MONTSERRAT',
 'NEW CALEDONIA',
 'NIUE',
 'SAINT HELENA, ASCENSION, AND TRISTAN DA CUNHA',
 'SAINT PIERRE AND MIQUELON',
 'SOMALIA',
 'SOUTH AFRICA',
 'SYRIA',
 'TAIWAN',
 'VENEZUELA',
 'WAKE ISLAND'}

In [33]:
set(forest.Country)-set(co2.name)

{'ANDORRA',
 'CURACAO',
 'ISLE OF MAN',
 'LIECHTENSTEIN',
 'MONACO',
 'PALAU',
 'SAN MARINO'}

Apparently, the data is not available from every country. So, let's just continue:

In [34]:
# the default is inner merge
cia=co2.merge(forest,left_on='name',right_on='Country')
cia

,name,co2,co2_date,region,Country,ForestRev_gdp,ForestRev_date
0,CHINA,1.077325e+10,2019,EAST AND SOUTHEAST ASIA,CHINA,0.08,2018
1,UNITED STATES,5.144361e+09,2019,NORTH AMERICA,UNITED STATES,0.04,2018
2,INDIA,2.314738e+09,2019,SOUTH ASIA,INDIA,0.14,2018
3,RUSSIA,1.848070e+09,2019,CENTRAL ASIA,RUSSIA,0.29,2018
4,JAPAN,1.103234e+09,2019,EAST AND SOUTHEAST ASIA,JAPAN,0.02,2018
...,...,...,...,...,...,...,...
192,TONGA,1.710000e+05,2019,AUSTRALIA AND OCEANIA,TONGA,0.03,2018
193,KIRIBATI,7.600000e+04,2019,AUSTRALIA AND OCEANIA,KIRIBATI,0.04,2018
194,NAURU,6.600000e+04,2019,AUSTRALIA AND OCEANIA,NAURU,0.00,2018
195,NORTHERN MARIANA ISLANDS,0.000000e+00,2019,AUSTRALIA AND OCEANIA,NORTHERN MARIANA ISLANDS,0.00,2018


Let's bring back the data on fragility, but just for the year 2019:

In [35]:
import os

linkFromGit='https://github.com/CienciaDeDatosEspacial/reshaping/raw/main/Fragility_cleaner.csv'
#read in:
FragilityAll=pd.read_csv(linkFromGit,index_col=False)


#subset
fragile2019=FragilityAll.loc[FragilityAll.Year==2019,:"Total"].copy()

# see
fragile2019

,Country,Year,Total
1068,YEMEN,2019,113.5
1069,SOMALIA,2019,112.3
1070,SOUTH SUDAN,2019,112.2
1071,SYRIA,2019,111.5
1072,CONGO DEMOCRATIC REPUBLIC,2019,110.2
...,...,...,...
1241,AUSTRALIA,2019,19.7
1242,DENMARK,2019,19.5
1243,SWITZERLAND,2019,18.7
1244,NORWAY,2019,18.0


We will practice **fuzzy merging** now.

In [36]:
# Countries in 'cia' but NOT in 'fragile2019' 
OnlyCia=set(cia.Country)-set(fragile2019.Country)
OnlyCia

{'AMERICAN SAMOA',
 'ARUBA',
 'BAHAMAS, THE',
 'BRUNEI',
 'BURMA',
 'CABO VERDE',
 'CAYMAN ISLANDS',
 'CONGO, DEMOCRATIC REPUBLIC OF THE',
 'CONGO, REPUBLIC OF THE',
 'CZECHIA',
 'DOMINICA',
 'FAROE ISLANDS',
 'GAMBIA, THE',
 'GAZA STRIP',
 'GREENLAND',
 'GUAM',
 'GUINEA-BISSAU',
 'HONG KONG',
 'KIRIBATI',
 'KOREA, SOUTH',
 'KOSOVO',
 'KYRGYZSTAN',
 'MACAU',
 'MARSHALL ISLANDS',
 'MICRONESIA, FEDERATED STATES OF',
 'NAURU',
 'NORTH MACEDONIA',
 'NORTHERN MARIANA ISLANDS',
 'PUERTO RICO',
 'SAINT KITTS AND NEVIS',
 'SAINT LUCIA',
 'SAINT VINCENT AND THE GRENADINES',
 'SLOVAKIA',
 'TONGA',
 'TURKEY (TURKIYE)',
 'TURKS AND CAICOS ISLANDS',
 'TUVALU',
 'VANUATU',
 'VIRGIN ISLANDS',
 'WEST BANK'}

In [37]:
# Countris in 'fragile2019' but NOT in 'cia' 
OnlyFragile=set(fragile2019.Country)-set(cia.Country)
OnlyFragile

{'BAHAMAS',
 'BRUNEI DARUSSALAM',
 'CAPE VERDE',
 'CONGO DEMOCRATIC REPUBLIC',
 'CONGO REPUBLIC',
 'CZECH REPUBLIC',
 'ERITREA',
 'GAMBIA',
 'GUINEA BISSAU',
 'KYRGYZ REPUBLIC',
 'MACEDONIA',
 'MICRONESIA',
 'MYANMAR',
 'NORTH KOREA',
 'SLOVAK REPUBLIC',
 'SOMALIA',
 'SOUTH AFRICA',
 'SOUTH KOREA',
 'SYRIA',
 'TURKEY',
 'VENEZUELA'}

Here, we should try to find the what countries in _OnlyFragile_ may match the ones in _OnlyCia_. We need to use the **fuzzy merge** approach (please install **thefuzz** if not previously installed):

In [38]:
# !pip install thefuzz

In [39]:
from thefuzz import process as fz

# take a country from OnlyFragile
# look for a country in OnlyCia and return the most similar
[(c,fz.extractOne(c,OnlyFragile )) for c in sorted(OnlyCia)]

[('AMERICAN SAMOA', ('SOUTH AFRICA', 59)),
 ('ARUBA', ('GAMBIA', 55)),
 ('BAHAMAS, THE', ('BAHAMAS', 90)),
 ('BRUNEI', ('BRUNEI DARUSSALAM', 90)),
 ('BURMA', ('BAHAMAS', 50)),
 ('CABO VERDE', ('CAPE VERDE', 80)),
 ('CAYMAN ISLANDS', ('MACEDONIA', 50)),
 ('CONGO, DEMOCRATIC REPUBLIC OF THE', ('CONGO DEMOCRATIC REPUBLIC', 95)),
 ('CONGO, REPUBLIC OF THE', ('CZECH REPUBLIC', 86)),
 ('CZECHIA', ('CZECH REPUBLIC', 64)),
 ('DOMINICA', ('MACEDONIA', 59)),
 ('FAROE ISLANDS', ('MICRONESIA', 43)),
 ('GAMBIA, THE', ('GAMBIA', 90)),
 ('GAZA STRIP', ('SYRIA', 54)),
 ('GREENLAND', ('VENEZUELA', 44)),
 ('GUAM', ('GAMBIA', 68)),
 ('GUINEA-BISSAU', ('GUINEA BISSAU', 100)),
 ('HONG KONG', ('CONGO DEMOCRATIC REPUBLIC', 50)),
 ('KIRIBATI', ('SYRIA', 54)),
 ('KOREA, SOUTH', ('SOUTH KOREA', 95)),
 ('KOSOVO', ('SLOVAK REPUBLIC', 43)),
 ('KYRGYZSTAN', ('KYRGYZ REPUBLIC', 54)),
 ('MACAU', ('MACEDONIA', 57)),
 ('MARSHALL ISLANDS', ('SOMALIA', 51)),
 ('MICRONESIA, FEDERATED STATES OF', ('MICRONESIA', 90)),
 ('NA

Above you have found _some_ good matches. Let's keep the best ones:

In [40]:
[(c,fz.extractOne(c, OnlyFragile)) for c in sorted(OnlyCia)
 if fz.extractOne(c, OnlyFragile)[1]>86]

[('BAHAMAS, THE', ('BAHAMAS', 90)),
 ('BRUNEI', ('BRUNEI DARUSSALAM', 90)),
 ('CONGO, DEMOCRATIC REPUBLIC OF THE', ('CONGO DEMOCRATIC REPUBLIC', 95)),
 ('GAMBIA, THE', ('GAMBIA', 90)),
 ('GUINEA-BISSAU', ('GUINEA BISSAU', 100)),
 ('KOREA, SOUTH', ('SOUTH KOREA', 95)),
 ('MICRONESIA, FEDERATED STATES OF', ('MICRONESIA', 90)),
 ('NORTH MACEDONIA', ('MACEDONIA', 90)),
 ('TURKEY (TURKIYE)', ('TURKEY', 90))]

Once you have good matches, you have to create dictionary like this:

In [41]:
changesCIA1={c:fz.extractOne(c, OnlyFragile)[0] 
                 for c in sorted(OnlyCia)
                 if fz.extractOne(c, OnlyFragile)[1] >86}
#dict of matches
changesCIA1

{'BAHAMAS, THE': 'BAHAMAS',
 'BRUNEI': 'BRUNEI DARUSSALAM',
 'CONGO, DEMOCRATIC REPUBLIC OF THE': 'CONGO DEMOCRATIC REPUBLIC',
 'GAMBIA, THE': 'GAMBIA',
 'GUINEA-BISSAU': 'GUINEA BISSAU',
 'KOREA, SOUTH': 'SOUTH KOREA',
 'MICRONESIA, FEDERATED STATES OF': 'MICRONESIA',
 'NORTH MACEDONIA': 'MACEDONIA',
 'TURKEY (TURKIYE)': 'TURKEY'}

You can use that dict for the replacements:

In [42]:
cia.replace(to_replace={'Country':changesCIA1},inplace=True)

This process can be done a few more times, and you can recover more rows for the merging process. Let's see:

In [43]:
# second try
OnlyCia=set(cia.Country)-set(fragile2019.Country)
OnlyFragile=set(fragile2019.Country)-set(cia.Country)
[(c,fz.extractOne(c,OnlyFragile )) for c in sorted(OnlyCia)]

[('AMERICAN SAMOA', ('SOUTH AFRICA', 59)),
 ('ARUBA', ('SLOVAK REPUBLIC', 40)),
 ('BURMA', ('SYRIA', 40)),
 ('CABO VERDE', ('CAPE VERDE', 80)),
 ('CAYMAN ISLANDS', ('SOMALIA', 39)),
 ('CONGO, REPUBLIC OF THE', ('CZECH REPUBLIC', 86)),
 ('CZECHIA', ('CZECH REPUBLIC', 64)),
 ('DOMINICA', ('SOMALIA', 53)),
 ('FAROE ISLANDS', ('SLOVAK REPUBLIC', 41)),
 ('GAZA STRIP', ('SYRIA', 54)),
 ('GREENLAND', ('VENEZUELA', 44)),
 ('GUAM', ('MYANMAR', 45)),
 ('HONG KONG', ('NORTH KOREA', 40)),
 ('KIRIBATI', ('SYRIA', 54)),
 ('KOSOVO', ('SLOVAK REPUBLIC', 43)),
 ('KYRGYZSTAN', ('KYRGYZ REPUBLIC', 54)),
 ('MACAU', ('SOMALIA', 50)),
 ('MARSHALL ISLANDS', ('SOMALIA', 51)),
 ('NAURU', ('MYANMAR', 50)),
 ('NORTHERN MARIANA ISLANDS', ('NORTH KOREA', 62)),
 ('PUERTO RICO', ('SOUTH AFRICA', 52)),
 ('SAINT KITTS AND NEVIS', ('NORTH KOREA', 40)),
 ('SAINT LUCIA', ('SOMALIA', 56)),
 ('SAINT VINCENT AND THE GRENADINES', ('ERITREA', 51)),
 ('SLOVAKIA', ('SLOVAK REPUBLIC', 74)),
 ('TONGA', ('CONGO REPUBLIC', 54)),
 (

In [44]:
# in the opposite direction
[(f,fz.extractOne(f, OnlyCia)) for f in sorted(OnlyFragile)]

[('CAPE VERDE', ('CABO VERDE', 80)),
 ('CONGO REPUBLIC', ('CONGO, REPUBLIC OF THE', 86)),
 ('CZECH REPUBLIC', ('CONGO, REPUBLIC OF THE', 86)),
 ('ERITREA', ('SAINT VINCENT AND THE GRENADINES', 51)),
 ('KYRGYZ REPUBLIC', ('CONGO, REPUBLIC OF THE', 66)),
 ('MYANMAR', ('NORTHERN MARIANA ISLANDS', 51)),
 ('NORTH KOREA', ('NORTHERN MARIANA ISLANDS', 62)),
 ('SLOVAK REPUBLIC', ('SLOVAKIA', 74)),
 ('SOMALIA', ('SLOVAKIA', 67)),
 ('SOUTH AFRICA', ('AMERICAN SAMOA', 59)),
 ('SYRIA', ('KIRIBATI', 54)),
 ('VENEZUELA', ('VANUATU', 50))]

In [45]:
troublesome={'CONGO REPUBLIC':'CONGO, REPUBLIC OF THE','KYRGYZ REPUBLIC':'KYRGYZSTAN'}

fragile2019.replace(to_replace={'Country':troublesome},inplace=True)

In [46]:
# third try
OnlyCia=set(cia.Country)-set(fragile2019.Country)
OnlyFragile=set(fragile2019.Country)-set(cia.Country)
[(f,fz.extractOne(f, OnlyCia)) for f in sorted(OnlyFragile)]


[('CAPE VERDE', ('CABO VERDE', 80)),
 ('CZECH REPUBLIC', ('CZECHIA', 64)),
 ('ERITREA', ('SAINT VINCENT AND THE GRENADINES', 51)),
 ('MYANMAR', ('NORTHERN MARIANA ISLANDS', 51)),
 ('NORTH KOREA', ('NORTHERN MARIANA ISLANDS', 62)),
 ('SLOVAK REPUBLIC', ('SLOVAKIA', 74)),
 ('SOMALIA', ('SLOVAKIA', 67)),
 ('SOUTH AFRICA', ('AMERICAN SAMOA', 59)),
 ('SYRIA', ('KIRIBATI', 54)),
 ('VENEZUELA', ('VANUATU', 50))]

In [47]:
# second dict of changes
# select a different threshold
changesFragile1={f:fz.extractOne(f, OnlyCia)[0] 
                 for f in sorted(OnlyFragile)
                 if fz.extractOne(f, OnlyCia)[1] in([80,64,74])}

#dict of matches
changesFragile1

{'CAPE VERDE': 'CABO VERDE',
 'CZECH REPUBLIC': 'CZECHIA',
 'SLOVAK REPUBLIC': 'SLOVAKIA'}

In [48]:
# make the changes
fragile2019.replace(to_replace={'Country':changesFragile1},inplace=True)

In [49]:
# next try
OnlyCia=set(cia.Country)-set(fragile2019.Country)
OnlyFragile=set(fragile2019.Country)-set(cia.Country)
[(f,fz.extractOne(f, OnlyCia)) for f in sorted(OnlyFragile)]

[('ERITREA', ('SAINT VINCENT AND THE GRENADINES', 51)),
 ('MYANMAR', ('NORTHERN MARIANA ISLANDS', 51)),
 ('NORTH KOREA', ('NORTHERN MARIANA ISLANDS', 62)),
 ('SOMALIA', ('NORTHERN MARIANA ISLANDS', 61)),
 ('SOUTH AFRICA', ('AMERICAN SAMOA', 59)),
 ('SYRIA', ('KIRIBATI', 54)),
 ('VENEZUELA', ('VANUATU', 50))]

We will stop here. Then,

In [50]:
fragilecia=fragile2019.merge(cia) #merge on Country
fragilecia

,Country,Year,Total,name,co2,co2_date,region,ForestRev_gdp,ForestRev_date
0,YEMEN,2019,113.5,YEMEN,10158000.0,2019,MIDDLE EAST,0.04,2018
1,SOUTH SUDAN,2019,112.2,SOUTH SUDAN,1778000.0,2019,AFRICA,2.65,2015
2,CONGO DEMOCRATIC REPUBLIC,2019,110.2,"CONGO, DEMOCRATIC REPUBLIC OF THE",2653000.0,2019,AFRICA,8.72,2018
3,CENTRAL AFRICAN REPUBLIC,2019,108.9,CENTRAL AFRICAN REPUBLIC,285000.0,2019,AFRICA,8.99,2018
4,CHAD,2019,108.5,CHAD,1771000.0,2019,AFRICA,3.81,2018
...,...,...,...,...,...,...,...,...,...
166,AUSTRALIA,2019,19.7,AUSTRALIA,417870000.0,2019,AUSTRALIA AND OCEANIA,0.13,2018
167,DENMARK,2019,19.5,DENMARK,33850000.0,2019,EUROPE,0.02,2018
168,SWITZERLAND,2019,18.7,SWITZERLAND,38739000.0,2019,EUROPE,0.01,2018
169,NORWAY,2019,18.0,NORWAY,36731000.0,2019,EUROPE,0.05,2018


In [51]:
#checking:
fragilecia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         171 non-null    object 
 1   Year            171 non-null    int64  
 2   Total           171 non-null    float64
 3   name            171 non-null    object 
 4   co2             171 non-null    float64
 5   co2_date        171 non-null    int64  
 6   region          171 non-null    object 
 7   ForestRev_gdp   171 non-null    float64
 8   ForestRev_date  171 non-null    int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 12.2+ KB


Merging is a key process for producing analytics. So, it is always good to add some 'standard' information to avoid the need of fuzzy merging. See this data table


In [52]:
isoLink='https://github.com/CienciaDeDatosEspacial/dataSets/raw/main/isodata.csv'
isoCodes=pd.read_csv(isoLink)
isoCodes.head()

,Countryname,Officialstatename,InternetccTLD,iso2,iso3
0,Afghanistan,The Islamic Republic of Afghanistan,.af,AF,AFG
1,Åland Islands,Åland,.ax,AX,ALA
2,Albania,The Republic of Albania,.al,AL,ALB
3,Algeria,The People's Democratic Republic of Algeria,.dz,DZ,DZA
4,American Samoa,The Territory of American Samoa,.as,AS,ASM


We should add the **ISO** columns to our recent merged data frame:

In [53]:
# key columns are not spelled the same:
isoCodes.Countryname=isoCodes.Countryname.str.upper()
isoCodes.merge(fragilecia,left_on='Countryname',right_on='Country')

,Countryname,Officialstatename,InternetccTLD,iso2,iso3,Country,Year,Total,name,co2,co2_date,region,ForestRev_gdp,ForestRev_date
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,AFGHANISTAN,2019,105.0,AFGHANISTAN,7893000.0,2019,SOUTH ASIA,0.20,2018
1,ALBANIA,The Republic of Albania,.al,AL,ALB,ALBANIA,2019,58.9,ALBANIA,3794000.0,2019,EUROPE,0.18,2018
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,ALGERIA,2019,75.4,ALGERIA,151633000.0,2019,AFRICA,0.10,2018
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,ANGOLA,2019,87.8,ANGOLA,19362000.0,2019,AFRICA,0.36,2018
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,ANTIGUA AND BARBUDA,2019,54.4,ANTIGUA AND BARBUDA,729000.0,2019,CENTRAL AMERICA AND THE CARIBBEAN,0.00,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,URUGUAY,The Oriental Republic of Uruguay,.uy,UY,URY,URUGUAY,2019,34.0,URUGUAY,6450000.0,2019,SOUTH AMERICA,1.56,2018
141,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,UZBEKISTAN,2019,75.7,UZBEKISTAN,102965000.0,2019,CENTRAL ASIA,0.00,2018
142,YEMEN,The Republic of Yemen,.ye,YE,YEM,YEMEN,2019,113.5,YEMEN,10158000.0,2019,MIDDLE EAST,0.04,2018
143,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,ZAMBIA,2019,85.7,ZAMBIA,6798000.0,2019,AFRICA,4.45,2018


We have lost several countries, then we redo the fuzzy merge:

In [54]:
onlyFrcia=set(fragilecia.Country)-set(isoCodes.Countryname)
onlyISO=set(isoCodes.Countryname)-set(fragilecia.Country)

[(f,fz.extractOne(f, onlyISO)) for f in sorted(onlyFrcia)]

[('BAHAMAS', ('BAHAMAS (THE)', 90)),
 ('BOLIVIA', ('BOLIVIA (PLURINATIONAL STATE OF)', 90)),
 ('CENTRAL AFRICAN REPUBLIC', ('CENTRAL AFRICAN REPUBLIC (THE)', 95)),
 ('COMOROS', ('COMOROS (THE)', 90)),
 ('CONGO DEMOCRATIC REPUBLIC', ('CONGO (THE DEMOCRATIC REPUBLIC OF THE)', 95)),
 ('CONGO, REPUBLIC OF THE', ('NIGER (THE)', 86)),
 ("COTE D'IVOIRE", ("CÔTE D'IVOIRE", 96)),
 ('DOMINICAN REPUBLIC', ('DOMINICAN REPUBLIC (THE)', 95)),
 ('GAMBIA', ('GAMBIA (THE)', 90)),
 ('GUINEA BISSAU', ('GUINEA-BISSAU', 100)),
 ('IRAN', ('IRAN (ISLAMIC REPUBLIC OF)', 90)),
 ('LAOS', ('ANGUILLA', 60)),
 ('MACEDONIA', ('NORTH MACEDONIA', 90)),
 ('MICRONESIA', ('MICRONESIA (FEDERATED STATES OF)', 90)),
 ('MOLDOVA', ('MOLDOVA (THE REPUBLIC OF)', 90)),
 ('NETHERLANDS', ('NETHERLANDS (THE)', 95)),
 ('NIGER', ('NIGER (THE)', 90)),
 ('PHILIPPINES', ('PHILIPPINES (THE)', 95)),
 ('RUSSIA', ('RUSSIAN FEDERATION (THE)', 90)),
 ('SOUTH KOREA', ('KOREA (THE REPUBLIC OF)', 86)),
 ('SUDAN', ('SUDAN (THE)', 90)),
 ('TANZAN

Prepare changes:

In [55]:
# first change
changesFrcia1={f:fz.extractOne(f, onlyISO)[0] 
                 for f in sorted(onlyFrcia)
                 if fz.extractOne(f, onlyISO)[1] >90}
#dict of matches
changesFrcia1

{'CENTRAL AFRICAN REPUBLIC': 'CENTRAL AFRICAN REPUBLIC (THE)',
 'CONGO DEMOCRATIC REPUBLIC': 'CONGO (THE DEMOCRATIC REPUBLIC OF THE)',
 "COTE D'IVOIRE": "CÔTE D'IVOIRE",
 'DOMINICAN REPUBLIC': 'DOMINICAN REPUBLIC (THE)',
 'GUINEA BISSAU': 'GUINEA-BISSAU',
 'NETHERLANDS': 'NETHERLANDS (THE)',
 'PHILIPPINES': 'PHILIPPINES (THE)',
 'UNITED ARAB EMIRATES': 'UNITED ARAB EMIRATES (THE)',
 'VIETNAM': 'VIET NAM'}

In [56]:
# make changes
fragilecia.replace(to_replace={'Country':changesFrcia1},inplace=True)

Second iteration

In [57]:
onlyFrcia=set(fragilecia.Country)-set(isoCodes.Countryname)
onlyISO=set(isoCodes.Countryname)-set(fragilecia.Country)

[(f,fz.extractOne(f, onlyISO)) for f in sorted(onlyFrcia)]

[('BAHAMAS', ('BAHAMAS (THE)', 90)),
 ('BOLIVIA', ('BOLIVIA (PLURINATIONAL STATE OF)', 90)),
 ('COMOROS', ('COMOROS (THE)', 90)),
 ('CONGO, REPUBLIC OF THE', ('NIGER (THE)', 86)),
 ('GAMBIA', ('GAMBIA (THE)', 90)),
 ('IRAN', ('IRAN (ISLAMIC REPUBLIC OF)', 90)),
 ('LAOS', ('ANGUILLA', 60)),
 ('MACEDONIA', ('NORTH MACEDONIA', 90)),
 ('MICRONESIA', ('MICRONESIA (FEDERATED STATES OF)', 90)),
 ('MOLDOVA', ('MOLDOVA (THE REPUBLIC OF)', 90)),
 ('NIGER', ('NIGER (THE)', 90)),
 ('RUSSIA', ('RUSSIAN FEDERATION (THE)', 90)),
 ('SOUTH KOREA', ('KOREA (THE REPUBLIC OF)', 86)),
 ('SUDAN', ('SUDAN (THE)', 90)),
 ('TANZANIA', ('TANZANIA, THE UNITED REPUBLIC OF', 90)),
 ('UNITED KINGDOM',
  ('UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)', 90)),
 ('UNITED STATES', ('UNITED STATES MINOR OUTLYING ISLANDS (THE)', 90))]

This second iteration gives weird results. Let's use a different function to get more than one result:

In [58]:
onlyFrcia=set(fragilecia.Country)-set(isoCodes.Countryname)
onlyISO=set(isoCodes.Countryname)-set(fragilecia.Country)

[(f,fz.extract(f, onlyISO)) for f in sorted(onlyFrcia)]

[('BAHAMAS',
  [('BAHAMAS (THE)', 90),
   ('WESTERN SAHARA', 56),
   ('GUAM', 45),
   ('MYANMAR', 43),
   ('BERMUDA', 43)]),
 ('BOLIVIA',
  [('BOLIVIA (PLURINATIONAL STATE OF)', 90),
   ('VENEZUELA (BOLIVARIAN REPUBLIC OF)', 77),
   ('SOMALIA', 57),
   ('DOMINICA', 53),
   ('MOLDOVA (THE REPUBLIC OF)', 51)]),
 ('COMOROS',
  [('COMOROS (THE)', 90),
   ('COCOS (KEELING) ISLANDS (THE)', 51),
   ('UNITED STATES MINOR OUTLYING ISLANDS (THE)', 51),
   ('COOK ISLANDS (THE)', 51),
   ('MICRONESIA (FEDERATED STATES OF)', 51)]),
 ('CONGO, REPUBLIC OF THE',
  [('NIGER (THE)', 86),
   ('SUDAN (THE)', 86),
   ('UNITED STATES MINOR OUTLYING ISLANDS (THE)', 86),
   ('BAHAMAS (THE)', 86),
   ('COMOROS (THE)', 86)]),
 ('GAMBIA',
  [('GAMBIA (THE)', 90),
   ('GUAM', 68),
   ('ARUBA', 55),
   ('SOMALIA', 46),
   ('MYANMAR', 46)]),
 ('IRAN',
  [('IRAN (ISLAMIC REPUBLIC OF)', 90),
   ('RUSSIAN FEDERATION (THE)', 68),
   ('NORTHERN MARIANA ISLANDS (THE)', 68),
   ('PITCAIRN', 68),
   ('SYRIAN ARAB REPUBLIC 

In [59]:
# remember you can use this for a particular case:
isoCodes.loc[isoCodes.Countryname.str.contains('LAO|KOREA|CONGO|GAZA|UNITED')]

,Countryname,Officialstatename,InternetccTLD,iso2,iso3
50,CONGO (THE DEMOCRATIC REPUBLIC OF THE),The Democratic Republic of the Congo,.cd,CD,COD
51,CONGO (THE),The Republic of the Congo,.cg,CG,COG
118,KOREA (THE DEMOCRATIC PEOPLE'S REPUBLIC OF),The Democratic People's Republic of Korea,.kp,KP,PRK
119,KOREA (THE REPUBLIC OF),The Republic of Korea,.kr,KR,KOR
122,LAO PEOPLE'S DEMOCRATIC REPUBLIC (THE),The Lao People's Democratic Republic,.la,LA,LAO
219,"TANZANIA, THE UNITED REPUBLIC OF",The United Republic of Tanzania,.tz,TZ,TZA
233,UNITED ARAB EMIRATES (THE),The United Arab Emirates,.ae,AE,ARE
234,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,The United Kingdom of Great Britain and Northe...,.uk,GB,GBR
235,UNITED STATES MINOR OUTLYING ISLANDS (THE),"Baker Island, Howland Island, Jarvis Island, J...",NaN,UM,UMI
236,UNITED STATES OF AMERICA (THE),The United States of America,.us,US,USA


Then, just prepare manual changes:

In [60]:
manualChanges={'CONGO, REPUBLIC OF THE':'CONGO (THE)',
 'KOREA, SOUTH':'KOREA (THE REPUBLIC OF)',
'LAOS':"LAO PEOPLE'S DEMOCRATIC REPUBLIC (THE)",'UNITED STATES':'UNITED STATES OF AMERICA (THE)'}

fragilecia.replace(to_replace={'Country':manualChanges},inplace=True)

In [61]:
onlyFrcia=set(fragilecia.Country)-set(isoCodes.Countryname)
onlyISO=set(isoCodes.Countryname)-set(fragilecia.Country)

[(f,fz.extractOne(f, onlyISO)) for f in sorted(onlyFrcia)]

[('BAHAMAS', ('BAHAMAS (THE)', 90)),
 ('BOLIVIA', ('BOLIVIA (PLURINATIONAL STATE OF)', 90)),
 ('COMOROS', ('COMOROS (THE)', 90)),
 ('GAMBIA', ('GAMBIA (THE)', 90)),
 ('IRAN', ('IRAN (ISLAMIC REPUBLIC OF)', 90)),
 ('MACEDONIA', ('NORTH MACEDONIA', 90)),
 ('MICRONESIA', ('MICRONESIA (FEDERATED STATES OF)', 90)),
 ('MOLDOVA', ('MOLDOVA (THE REPUBLIC OF)', 90)),
 ('NIGER', ('NIGER (THE)', 90)),
 ('RUSSIA', ('RUSSIAN FEDERATION (THE)', 90)),
 ('SOUTH KOREA', ('KOREA (THE REPUBLIC OF)', 86)),
 ('SUDAN', ('SUDAN (THE)', 90)),
 ('TANZANIA', ('TANZANIA, THE UNITED REPUBLIC OF', 90)),
 ('UNITED KINGDOM',
  ('UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)', 90))]

Then,

In [62]:
# last change
changesFrcia2={f:fz.extractOne(f, onlyISO)[0] 
                 for f in sorted(onlyFrcia)}
#dict of matches
changesFrcia2

{'BAHAMAS': 'BAHAMAS (THE)',
 'BOLIVIA': 'BOLIVIA (PLURINATIONAL STATE OF)',
 'COMOROS': 'COMOROS (THE)',
 'GAMBIA': 'GAMBIA (THE)',
 'IRAN': 'IRAN (ISLAMIC REPUBLIC OF)',
 'MACEDONIA': 'NORTH MACEDONIA',
 'MICRONESIA': 'MICRONESIA (FEDERATED STATES OF)',
 'MOLDOVA': 'MOLDOVA (THE REPUBLIC OF)',
 'NIGER': 'NIGER (THE)',
 'RUSSIA': 'RUSSIAN FEDERATION (THE)',
 'SOUTH KOREA': 'KOREA (THE REPUBLIC OF)',
 'SUDAN': 'SUDAN (THE)',
 'TANZANIA': 'TANZANIA, THE UNITED REPUBLIC OF',
 'UNITED KINGDOM': 'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)'}

In [63]:
fragilecia.replace(to_replace={'Country':changesFrcia2},inplace=True)

#check

onlyFrcia=set(fragilecia.Country)-set(isoCodes.Countryname)
onlyISO=set(isoCodes.Countryname)-set(fragilecia.Country)

[(f,fz.extractOne(f, onlyISO)) for f in sorted(onlyFrcia)]

[]

In [64]:
# now:

fragciaiso=isoCodes.merge(fragilecia,left_on='Countryname',right_on='Country')
fragciaiso.head()

,Countryname,Officialstatename,InternetccTLD,iso2,iso3,Country,Year,Total,name,co2,co2_date,region,ForestRev_gdp,ForestRev_date
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,AFGHANISTAN,2019,105.0,AFGHANISTAN,7893000.0,2019,SOUTH ASIA,0.20,2018
1,ALBANIA,The Republic of Albania,.al,AL,ALB,ALBANIA,2019,58.9,ALBANIA,3794000.0,2019,EUROPE,0.18,2018
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,ALGERIA,2019,75.4,ALGERIA,151633000.0,2019,AFRICA,0.10,2018
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,ANGOLA,2019,87.8,ANGOLA,19362000.0,2019,AFRICA,0.36,2018
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,ANTIGUA AND BARBUDA,2019,54.4,ANTIGUA AND BARBUDA,729000.0,2019,CENTRAL AMERICA AND THE CARIBBEAN,0.00,2018


In [ ]:
fragciaiso.drop(columns=['Country','name'],inplace=True)
fragciaiso.rename(columns={'Countryname':"Country",'Year':'fragility_date','Total':'fragility'},inplace=True)
fragciaiso

Let's save what we have:

In [ ]:
fragciaiso.to_csv("FragilityCia_isos.csv", index=False)